# Stock Tracker - Percentage Change Analysis - Compiled

Simple stock tracker to monitor percentage changes from baseline (July 25th, 2025) to any target date.


In [1]:
# Stock Tickers Configuration
STOCK_TICKERS = [
    'AFRM',  # Affirm Holdings Inc
    'PSTG',  # Pure Storage Inc
    'BABA',  # Alibaba Group Holding Ltd (ADR)
    'AVGO',  # Broadcom Inc.
    'CRM',   # Salesforce, Inc.
    'ADSK',  # Autodesk, Inc.
]

# Use realistic past dates for actual data
BASELINE_DATE = '2025-08-29' 

# Historical Comparison - Compare performance to current date
future_date_str = '2025-09-09'


In [2]:
# Import libraries
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime, timedelta
import yfinance as yf
import time  # For retries
import logging  # Optional: For better logging

# Set up logging (replace print with logging for production)
logging.basicConfig(level=logging.INFO)

print("Libraries imported successfully!")  # Keeping print for consistency with original
print(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("Using Plotly for interactive visualizations! 📊")

# # Stock Tickers Configuration
# STOCK_TICKERS = [
#     'AMZN',   # Amazon.com, Inc.
#     'AMD',    # Advanced Micro Devices, Inc.
#     'META',   # Meta Platforms, Inc.
#     'BA',     # The Boeing Company
#     'AAPL',   # Apple Inc.
#     'CAT',    # Caterpillar Inc.
#     'JNJ',    # Johnson & Johnson
#     'RCL',    # Royal Caribbean Cruises Ltd.
#     'PH',     # Parker-Hannifin Corporation
#     'MU',     # Micron Technology, Inc.
#     'FSLR',   # First Solar, Inc.
#     'LLY'     # Eli Lilly and Company
# ]
# 
# # Use realistic past dates for actual data
# BASELINE_DATE = '2025-07-25'  # Changed to past date that exists
# 
# # Historical Comparison - Compare performance to current date
# future_date_str = datetime.now().strftime('%Y-%m-%d')  # Use current date for comparison

# Will store baseline prices after fetching

baseline_prices = {}

print("✅ Stock tickers configured successfully!")
print(f"📅 Baseline date: {BASELINE_DATE}")
print(f"📈 Tracking {len(STOCK_TICKERS)} stocks")

class StockTracker:
    def __init__(self, tickers: list[str], baseline_date: str):
        self.tickers = tickers
        self.baseline_date = baseline_date
        self.baseline_prices = self.fetch_stock_prices(baseline_date)

    def fetch_stock_prices(self, date_str: str | None = None, compare_to_baseline: bool = False) -> dict[str, float]:
        """
        Fetch current or historical prices for all stocks in self.tickers using batch download
        
        Parameters:
        - date_str: Optional date string in 'YYYY-MM-DD' format. If None, gets latest prices.
        - compare_to_baseline: If True, shows percentage change from baseline
        
        Returns:
        - dict: {ticker: price}
        """
        if date_str is None:
            date_str = datetime.now().strftime('%Y-%m-%d')
            print(f"📈 Fetching LATEST stock prices (around {date_str})")
        else:
            print(f"📈 Fetching stock prices for: {date_str}")
        
        print("=" * 60)
        
        fetched_prices = {}
        try:
            start_date = datetime.strptime(date_str, '%Y-%m-%d') - timedelta(days=7)  # Buffer for non-trading days
            end_date = datetime.strptime(date_str, '%Y-%m-%d') + timedelta(days=1)
            
            for attempt in range(3):  # Retry up to 3 times
                data = yf.download(self.tickers, start=start_date, end=end_date)['Close']
                if not data.empty:
                    break
                print(f"Retry {attempt + 1}/3: No data, waiting...")
                time.sleep(2)
            
            if data.empty:
                print("❌ No data available for the specified period.")
                return {}
            
            successful_fetches = 0
            for ticker in self.tickers:
                if ticker in data.columns and not data[ticker].dropna().empty:
                    price = round(data[ticker].dropna().iloc[-1], 2)  # Latest available close
                    fetched_prices[ticker] = price
                    if compare_to_baseline and self.baseline_prices and ticker in self.baseline_prices:
                        baseline_price = self.baseline_prices[ticker]
                        if baseline_price is not None:
                            pct_change = ((price - baseline_price) / baseline_price) * 100
                            print(f"🔍 {ticker}: ${price:.2f} ({pct_change:+.1f}%)")
                        else:
                            print(f"🔍 {ticker}: ${price:.2f} (No baseline)")
                    else:
                        print(f"🔍 {ticker}: ${price:.2f}")
                    successful_fetches += 1
                else:
                    print(f"🔍 {ticker}: Failed")
            
            print(f"\n✅ Successfully fetched prices for {successful_fetches}/{len(self.tickers)} stocks")
            return fetched_prices
        except Exception as e:
            print(f"❌ Error in batch fetch: {e}")
            return {}

    def track_stocks(self, target_date: str, current_prices: dict[str, float]) -> pd.DataFrame | None:
        """
        Track stock performance from baseline to target date
        
        Parameters:
        - target_date: String (e.g., "2025-08-01")
        - current_prices: Dictionary with stock symbols and current prices
        
        Returns:
        - pd.DataFrame: Performance data, or None if no data
        """
        results = []
        
        print(f"📈 STOCK PERFORMANCE ANALYSIS")
        print(f"From: {self.baseline_date} → To: {target_date}")
        print("=" * 50)
        
        for symbol, current_price in current_prices.items():
            if symbol in self.baseline_prices and current_price is not None:
                baseline_price = self.baseline_prices[symbol]
                if baseline_price is not None:
                    price_change = current_price - baseline_price
                    pct_change = (price_change / baseline_price) * 100
                    
                    results.append({
                        'Symbol': symbol,
                        'Baseline': baseline_price,
                        'Current': current_price,
                        'Change': price_change,
                        'Percent_Change': pct_change
                    })
        
        if not results:
            print("No valid price data provided.")
            return None
        
        # Create DataFrame and sort by performance
        df = pd.DataFrame(results)
        df = df.sort_values('Percent_Change', ascending=False)
        
        # Display results
        print(f"{'Rank':<4} {'Symbol':<6} {'Baseline':<10} {'Current':<10} {'Change':<10} {'% Change':<10}")
        print("-" * 60)
        
        for i, (_, row) in enumerate(df.iterrows(), 1):
            print(f"{i:<4} {row['Symbol']:<6} ${row['Baseline']:<9.2f} ${row['Current']:<9.2f} "
                  f"${row['Change']:<9.2f} {row['Percent_Change']:<9.2f}%")
        
        # Summary
        avg_change = df['Percent_Change'].mean()
        print(f"\n🏆 Best Performer: {df.iloc[0]['Symbol']} ({df.iloc[0]['Percent_Change']:+.2f}%)")
        print(f"📉 Worst Performer: {df.iloc[-1]['Symbol']} ({df.iloc[-1]['Percent_Change']:+.2f}%)")
        print(f"📊 Average Change: {avg_change:+.2f}%")
        
        return df

    def create_performance_plot(self, target_date: str, current_prices: dict[str, float]) -> pd.DataFrame | None:
        """
        Create interactive visualization for stock performance using Plotly
        
        Parameters:
        - target_date: String (e.g., "2025-08-01")
        - current_prices: Dictionary with stock symbols and current prices
        
        Returns:
        - pd.DataFrame: Performance data, or None if no data
        """
        df = self.track_stocks(target_date, current_prices)
        
        if df is None:
            return None
        
        # Sort x-axis by performance
        df = df.sort_values('Percent_Change', ascending=False)
        
        # Calculate average change
        avg_change = df['Percent_Change'].mean()
        
        # Create color scheme based on performance
        colors = ['#00CC44' if x > 0 else '#FF4444' for x in df['Percent_Change']]
        
        # Create interactive bar chart
        fig = go.Figure()
        
        fig.add_trace(go.Bar(
            x=df['Symbol'],
            y=df['Percent_Change'],
            marker_color=colors,
            text=[f'{x:.1f}%' for x in df['Percent_Change']],
            textposition='outside',
            hovertemplate=
            '<b>%{x}</b><br>' +
            'Baseline: $%{customdata[0]:.2f}<br>' +
            'Current: $%{customdata[1]:.2f}<br>' +
            'Change: %{y:.2f}%<br>' +
            '<extra></extra>',
            customdata=list(zip(df['Baseline'], df['Current'])),
            showlegend=False
        ))
        
        # Add zero line
        fig.add_hline(y=0, line_dash="dash", line_color="black", opacity=0.5)
        
        # Add average change line
        fig.add_hline(
            y=avg_change,
            line_dash="dash",
            line_color="blue",
            annotation_text=f"Avg Change: {avg_change:.2f}%",
            annotation_position="bottom right"
        )
        
        # Add annotations for best and worst
        fig.add_annotation(x=df['Symbol'].iloc[0], y=df['Percent_Change'].iloc[0] + 5,
                           text="Best", showarrow=True, arrowhead=1)
        fig.add_annotation(x=df['Symbol'].iloc[-1], y=df['Percent_Change'].iloc[-1] - 5,
                           text="Worst", showarrow=True, arrowhead=1)
        
        # Update layout
        fig.update_layout(
            title={
                'text': f'📈 Stock Performance: {self.baseline_date} → {target_date}',
                'x': 0.5,
                'font': {'size': 20, 'family': 'Arial Black'}
            },
            xaxis_title="Stock Symbol",
            yaxis_title="Percentage Change (%)",
            height=600,
            width=1000,
            template="plotly_white",
            font=dict(size=12),
            hovermode='x',
            plot_bgcolor='rgba(240, 240, 240, 0.1)'
        )
        
        # Update axes
        fig.update_xaxes(tickangle=45, tickfont=dict(size=11), categoryorder='array', categoryarray=df['Symbol'].tolist())
        fig.update_yaxes(tickfont=dict(size=11), gridcolor='rgba(128, 128, 128, 0.3)')
        
        # Show the interactive plot
        fig.show()
        
        # Optional: Export
        # fig.write_html('performance.html')
        # fig.write_image('performance.png')
        
        return df

# Initialize tracker
tracker = StockTracker(STOCK_TICKERS, BASELINE_DATE)

print(f"\n📊 Baseline Prices ({BASELINE_DATE}):")
print(tracker.baseline_prices)

print("✅ Stock tracking class ready!")
print("📊 Interactive Plotly visualization ready!")

print(f"📊 HISTORICAL COMPARISON")
print(f"Baseline: {BASELINE_DATE} → Target: {future_date_str}")
print("=" * 50)

# Fetch prices for future date
future_prices = tracker.fetch_stock_prices(future_date_str, compare_to_baseline=True)

# DEMO: Interactive Plotly Visualizations
print("\n🎯 Historical Performance Analysis")
print("=" * 45)

print("Creating interactive Plotly charts...")
print(f"\nPerformance from {BASELINE_DATE} to {future_date_str}:")
demo_result = tracker.create_performance_plot(future_date_str, future_prices)


[*********************100%***********************]  6 of 6 completed
ERROR:yfinance:
6 Failed downloads:
ERROR:yfinance:['AFRM', 'ADSK', 'BABA', 'CRM', 'AVGO', 'PSTG']: SSLError('Failed to perform, curl: (35) TLS connect error: error:00000000:invalid library (0):OPENSSL_internal:invalid library (0). See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


Libraries imported successfully!
Date: 2025-09-16 12:31:49
Using Plotly for interactive visualizations! 📊
✅ Stock tickers configured successfully!
📅 Baseline date: 2025-08-29
📈 Tracking 6 stocks
📈 Fetching stock prices for: 2025-08-29
YF.download() has changed argument auto_adjust default to True
Retry 1/3: No data, waiting...


[*********************100%***********************]  6 of 6 completed
ERROR:yfinance:
6 Failed downloads:
ERROR:yfinance:['PSTG', 'CRM', 'BABA', 'AFRM', 'AVGO', 'ADSK']: SSLError('Failed to perform, curl: (35) TLS connect error: error:00000000:invalid library (0):OPENSSL_internal:invalid library (0). See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


Retry 2/3: No data, waiting...


[*********************100%***********************]  6 of 6 completed
ERROR:yfinance:
6 Failed downloads:
ERROR:yfinance:['CRM', 'PSTG', 'BABA', 'AFRM', 'ADSK', 'AVGO']: SSLError('Failed to perform, curl: (35) TLS connect error: error:00000000:invalid library (0):OPENSSL_internal:invalid library (0). See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


...

❌ No data available for the specified period.

📊 Baseline Prices (2025-08-29):
{}
✅ Stock tracking class ready!
📊 Interactive Plotly visualization ready!
📊 HISTORICAL COMPARISON
Baseline: 2025-08-29 → Target: 2025-09-09
📈 Fetching stock prices for: 2025-09-09
Retry 1/3: No data, waiting...


[*********************100%***********************]  6 of 6 completed
ERROR:yfinance:
6 Failed downloads:
ERROR:yfinance:['PSTG', 'BABA', 'AFRM', 'AVGO', 'ADSK', 'CRM']: SSLError('Failed to perform, curl: (35) TLS connect error: error:00000000:invalid library (0):OPENSSL_internal:invalid library (0). See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


Retry 2/3: No data, waiting...


[*********************100%***********************]  6 of 6 completed
ERROR:yfinance:
6 Failed downloads:
ERROR:yfinance:['CRM', 'PSTG', 'BABA', 'AVGO', 'ADSK', 'AFRM']: SSLError('Failed to perform, curl: (35) TLS connect error: error:00000000:invalid library (0):OPENSSL_internal:invalid library (0). See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


Retry 3/3: No data, waiting...
❌ No data available for the specified period.

🎯 Historical Performance Analysis
Creating interactive Plotly charts...

Performance from 2025-08-29 to 2025-09-09:
📈 STOCK PERFORMANCE ANALYSIS
From: 2025-08-29 → To: 2025-09-09
No valid price data provided.


In [3]:
# Ted's overnight picks

# For the first time my AI model gave me 5 picks all winners: 
# "NUKK went up by 9.86%. SOGP went up by 82.4%. IFRX went up by 9.38%. GMHS went up by 12.69%. DUO went up by 9.21%."

# The picks for today's rise: 10% (MLYS, MNKD, CYTK), 5% (UTHR, ULCC, BABA)

In [4]:
# Stock Tickers Configuration
STOCK_TICKERS = [
    'NUKK',  # Affirm Holdings Inc
    'SOGP',  # Pure Storage Inc
    'IFRX',  # Alibaba Group Holding Ltd (ADR)
    'GMHS',  # Broadcom Inc.
    'DUO'   # Salesforce, Inc.
]

# Use realistic past dates for actual data
BASELINE_DATE = '2025-09-01' 

# Historical Comparison - Compare performance to current date
future_date_str = '2025-09-06'

# Import libraries
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime, timedelta
import yfinance as yf
import time  # For retries
import logging  # Optional: For better logging

# Set up logging (replace print with logging for production)
logging.basicConfig(level=logging.INFO)

print("Libraries imported successfully!")  # Keeping print for consistency with original
print(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("Using Plotly for interactive visualizations! 📊")

baseline_prices = {}

print("✅ Stock tickers configured successfully!")
print(f"📅 Baseline date: {BASELINE_DATE}")
print(f"📈 Tracking {len(STOCK_TICKERS)} stocks")

class StockTracker:
    def __init__(self, tickers: list[str], baseline_date: str):
        self.tickers = tickers
        self.baseline_date = baseline_date
        self.baseline_prices = self.fetch_stock_prices(baseline_date)

    def fetch_stock_prices(self, date_str: str | None = None, compare_to_baseline: bool = False) -> dict[str, float]:
        """
        Fetch current or historical prices for all stocks in self.tickers using batch download
        
        Parameters:
        - date_str: Optional date string in 'YYYY-MM-DD' format. If None, gets latest prices.
        - compare_to_baseline: If True, shows percentage change from baseline
        
        Returns:
        - dict: {ticker: price}
        """
        if date_str is None:
            date_str = datetime.now().strftime('%Y-%m-%d')
            print(f"📈 Fetching LATEST stock prices (around {date_str})")
        else:
            print(f"📈 Fetching stock prices for: {date_str}")
        
        print("=" * 60)
        
        fetched_prices = {}
        try:
            start_date = datetime.strptime(date_str, '%Y-%m-%d') - timedelta(days=7)  # Buffer for non-trading days
            end_date = datetime.strptime(date_str, '%Y-%m-%d') + timedelta(days=1)
            
            for attempt in range(3):  # Retry up to 3 times
                data = yf.download(self.tickers, start=start_date, end=end_date)['Close']
                if not data.empty:
                    break
                print(f"Retry {attempt + 1}/3: No data, waiting...")
                time.sleep(2)
            
            if data.empty:
                print("❌ No data available for the specified period.")
                return {}
            
            successful_fetches = 0
            for ticker in self.tickers:
                if ticker in data.columns and not data[ticker].dropna().empty:
                    price = round(data[ticker].dropna().iloc[-1], 2)  # Latest available close
                    fetched_prices[ticker] = price
                    if compare_to_baseline and self.baseline_prices and ticker in self.baseline_prices:
                        baseline_price = self.baseline_prices[ticker]
                        if baseline_price is not None:
                            pct_change = ((price - baseline_price) / baseline_price) * 100
                            print(f"🔍 {ticker}: ${price:.2f} ({pct_change:+.1f}%)")
                        else:
                            print(f"🔍 {ticker}: ${price:.2f} (No baseline)")
                    else:
                        print(f"🔍 {ticker}: ${price:.2f}")
                    successful_fetches += 1
                else:
                    print(f"🔍 {ticker}: Failed")
            
            print(f"\n✅ Successfully fetched prices for {successful_fetches}/{len(self.tickers)} stocks")
            return fetched_prices
        except Exception as e:
            print(f"❌ Error in batch fetch: {e}")
            return {}

    def track_stocks(self, target_date: str, current_prices: dict[str, float]) -> pd.DataFrame | None:
        """
        Track stock performance from baseline to target date
        
        Parameters:
        - target_date: String (e.g., "2025-08-01")
        - current_prices: Dictionary with stock symbols and current prices
        
        Returns:
        - pd.DataFrame: Performance data, or None if no data
        """
        results = []
        
        print(f"📈 STOCK PERFORMANCE ANALYSIS")
        print(f"From: {self.baseline_date} → To: {target_date}")
        print("=" * 50)
        
        for symbol, current_price in current_prices.items():
            if symbol in self.baseline_prices and current_price is not None:
                baseline_price = self.baseline_prices[symbol]
                if baseline_price is not None:
                    price_change = current_price - baseline_price
                    pct_change = (price_change / baseline_price) * 100
                    
                    results.append({
                        'Symbol': symbol,
                        'Baseline': baseline_price,
                        'Current': current_price,
                        'Change': price_change,
                        'Percent_Change': pct_change
                    })
        
        if not results:
            print("No valid price data provided.")
            return None
        
        # Create DataFrame and sort by performance
        df = pd.DataFrame(results)
        df = df.sort_values('Percent_Change', ascending=False)
        
        # Display results
        print(f"{'Rank':<4} {'Symbol':<6} {'Baseline':<10} {'Current':<10} {'Change':<10} {'% Change':<10}")
        print("-" * 60)
        
        for i, (_, row) in enumerate(df.iterrows(), 1):
            print(f"{i:<4} {row['Symbol']:<6} ${row['Baseline']:<9.2f} ${row['Current']:<9.2f} "
                  f"${row['Change']:<9.2f} {row['Percent_Change']:<9.2f}%")
        
        # Summary
        avg_change = df['Percent_Change'].mean()
        print(f"\n🏆 Best Performer: {df.iloc[0]['Symbol']} ({df.iloc[0]['Percent_Change']:+.2f}%)")
        print(f"📉 Worst Performer: {df.iloc[-1]['Symbol']} ({df.iloc[-1]['Percent_Change']:+.2f}%)")
        print(f"📊 Average Change: {avg_change:+.2f}%")
        
        return df

    def create_performance_plot(self, target_date: str, current_prices: dict[str, float]) -> pd.DataFrame | None:
        """
        Create interactive visualization for stock performance using Plotly
        
        Parameters:
        - target_date: String (e.g., "2025-08-01")
        - current_prices: Dictionary with stock symbols and current prices
        
        Returns:
        - pd.DataFrame: Performance data, or None if no data
        """
        df = self.track_stocks(target_date, current_prices)
        
        if df is None:
            return None
        
        # Sort x-axis by performance
        df = df.sort_values('Percent_Change', ascending=False)
        
        # Calculate average change
        avg_change = df['Percent_Change'].mean()
        
        # Create color scheme based on performance
        colors = ['#00CC44' if x > 0 else '#FF4444' for x in df['Percent_Change']]
        
        # Create interactive bar chart
        fig = go.Figure()
        
        fig.add_trace(go.Bar(
            x=df['Symbol'],
            y=df['Percent_Change'],
            marker_color=colors,
            text=[f'{x:.1f}%' for x in df['Percent_Change']],
            textposition='outside',
            hovertemplate=
            '<b>%{x}</b><br>' +
            'Baseline: $%{customdata[0]:.2f}<br>' +
            'Current: $%{customdata[1]:.2f}<br>' +
            'Change: %{y:.2f}%<br>' +
            '<extra></extra>',
            customdata=list(zip(df['Baseline'], df['Current'])),
            showlegend=False
        ))
        
        # Add zero line
        fig.add_hline(y=0, line_dash="dash", line_color="black", opacity=0.5)
        
        # Add average change line
        fig.add_hline(
            y=avg_change,
            line_dash="dash",
            line_color="blue",
            annotation_text=f"Avg Change: {avg_change:.2f}%",
            annotation_position="bottom right"
        )
        
        # Add annotations for best and worst
        fig.add_annotation(x=df['Symbol'].iloc[0], y=df['Percent_Change'].iloc[0] + 5,
                           text="Best", showarrow=True, arrowhead=1)
        fig.add_annotation(x=df['Symbol'].iloc[-1], y=df['Percent_Change'].iloc[-1] - 5,
                           text="Worst", showarrow=True, arrowhead=1)
        
        # Update layout
        fig.update_layout(
            title={
                'text': f'📈 Stock Performance: {self.baseline_date} → {target_date}',
                'x': 0.5,
                'font': {'size': 20, 'family': 'Arial Black'}
            },
            xaxis_title="Stock Symbol",
            yaxis_title="Percentage Change (%)",
            height=600,
            width=1000,
            template="plotly_white",
            font=dict(size=12),
            hovermode='x',
            plot_bgcolor='rgba(240, 240, 240, 0.1)'
        )
        
        # Update axes
        fig.update_xaxes(tickangle=45, tickfont=dict(size=11), categoryorder='array', categoryarray=df['Symbol'].tolist())
        fig.update_yaxes(tickfont=dict(size=11), gridcolor='rgba(128, 128, 128, 0.3)')
        
        # Show the interactive plot
        fig.show()
        
        # Optional: Export
        # fig.write_html('performance.html')
        # fig.write_image('performance.png')
        
        return df

# Initialize tracker
tracker = StockTracker(STOCK_TICKERS, BASELINE_DATE)

print(f"\n📊 Baseline Prices ({BASELINE_DATE}):")
print(tracker.baseline_prices)

print("✅ Stock tracking class ready!")
print("📊 Interactive Plotly visualization ready!")

print(f"📊 HISTORICAL COMPARISON")
print(f"Baseline: {BASELINE_DATE} → Target: {future_date_str}")
print("=" * 50)

# Fetch prices for future date
future_prices = tracker.fetch_stock_prices(future_date_str, compare_to_baseline=True)

# DEMO: Interactive Plotly Visualizations
print("\n🎯 Historical Performance Analysis")
print("=" * 45)

print("Creating interactive Plotly charts...")
print(f"\nPerformance from {BASELINE_DATE} to {future_date_str}:")
demo_result = tracker.create_performance_plot(future_date_str, future_prices)



[*********************100%***********************]  5 of 5 completed
ERROR:yfinance:
5 Failed downloads:
ERROR:yfinance:['SOGP', 'GMHS', 'DUO', 'NUKK', 'IFRX']: SSLError('Failed to perform, curl: (35) TLS connect error: error:00000000:invalid library (0):OPENSSL_internal:invalid library (0). See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


Libraries imported successfully!
Date: 2025-09-16 12:32:02
Using Plotly for interactive visualizations! 📊
✅ Stock tickers configured successfully!
📅 Baseline date: 2025-09-01
📈 Tracking 5 stocks
📈 Fetching stock prices for: 2025-09-01
Retry 1/3: No data, waiting...


[*********************100%***********************]  5 of 5 completed
ERROR:yfinance:
5 Failed downloads:
ERROR:yfinance:['SOGP', 'GMHS', 'DUO', 'IFRX', 'NUKK']: SSLError('Failed to perform, curl: (35) TLS connect error: error:00000000:invalid library (0):OPENSSL_internal:invalid library (0). See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


Retry 2/3: No data, waiting...


[*********************100%***********************]  5 of 5 completed
ERROR:yfinance:
5 Failed downloads:
ERROR:yfinance:['SOGP', 'GMHS', 'NUKK', 'IFRX', 'DUO']: SSLError('Failed to perform, curl: (35) TLS connect error: error:00000000:invalid library (0):OPENSSL_internal:invalid library (0). See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


...

❌ No data available for the specified period.

📊 Baseline Prices (2025-09-01):
{}
✅ Stock tracking class ready!
📊 Interactive Plotly visualization ready!
📊 HISTORICAL COMPARISON
Baseline: 2025-09-01 → Target: 2025-09-06
📈 Fetching stock prices for: 2025-09-06
Retry 1/3: No data, waiting...


[*********************100%***********************]  5 of 5 completed
ERROR:yfinance:
5 Failed downloads:
ERROR:yfinance:['SOGP', 'GMHS', 'NUKK', 'IFRX', 'DUO']: SSLError('Failed to perform, curl: (35) TLS connect error: error:00000000:invalid library (0):OPENSSL_internal:invalid library (0). See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


Retry 2/3: No data, waiting...


[*********************100%***********************]  5 of 5 completed
ERROR:yfinance:
5 Failed downloads:
ERROR:yfinance:['SOGP', 'GMHS', 'NUKK', 'IFRX', 'DUO']: SSLError('Failed to perform, curl: (35) TLS connect error: error:00000000:invalid library (0):OPENSSL_internal:invalid library (0). See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


Retry 3/3: No data, waiting...
❌ No data available for the specified period.

🎯 Historical Performance Analysis
Creating interactive Plotly charts...

Performance from 2025-09-01 to 2025-09-06:
📈 STOCK PERFORMANCE ANALYSIS
From: 2025-09-01 → To: 2025-09-06
No valid price data provided.


In [5]:
# Stock Tickers Configuration
STOCK_TICKERS = [
    'MLYS',  # Affirm Holdings Inc
    'MNKD',  # Pure Storage Inc
    'CYTK',  # Alibaba Group Holding Ltd (ADR)
    'UTHR',  # Broadcom Inc.
    'ULCC',   # Salesforce, Inc.
    'BABA'
]

# Use realistic past dates for actual data
BASELINE_DATE = '2025-09-02' 

# Historical Comparison - Compare performance to current date
future_date_str = '2025-09-06'

# Import libraries
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime, timedelta
import yfinance as yf
import time  # For retries
import logging  # Optional: For better logging

# Set up logging (replace print with logging for production)
logging.basicConfig(level=logging.INFO)

print("Libraries imported successfully!")  # Keeping print for consistency with original
print(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("Using Plotly for interactive visualizations! 📊")

baseline_prices = {}

print("✅ Stock tickers configured successfully!")
print(f"📅 Baseline date: {BASELINE_DATE}")
print(f"📈 Tracking {len(STOCK_TICKERS)} stocks")

class StockTracker:
    def __init__(self, tickers: list[str], baseline_date: str):
        self.tickers = tickers
        self.baseline_date = baseline_date
        self.baseline_prices = self.fetch_stock_prices(baseline_date)

    def fetch_stock_prices(self, date_str: str | None = None, compare_to_baseline: bool = False) -> dict[str, float]:
        """
        Fetch current or historical prices for all stocks in self.tickers using batch download
        
        Parameters:
        - date_str: Optional date string in 'YYYY-MM-DD' format. If None, gets latest prices.
        - compare_to_baseline: If True, shows percentage change from baseline
        
        Returns:
        - dict: {ticker: price}
        """
        if date_str is None:
            date_str = datetime.now().strftime('%Y-%m-%d')
            print(f"📈 Fetching LATEST stock prices (around {date_str})")
        else:
            print(f"📈 Fetching stock prices for: {date_str}")
        
        print("=" * 60)
        
        fetched_prices = {}
        try:
            start_date = datetime.strptime(date_str, '%Y-%m-%d') - timedelta(days=7)  # Buffer for non-trading days
            end_date = datetime.strptime(date_str, '%Y-%m-%d') + timedelta(days=1)
            
            for attempt in range(3):  # Retry up to 3 times
                data = yf.download(self.tickers, start=start_date, end=end_date)['Close']
                if not data.empty:
                    break
                print(f"Retry {attempt + 1}/3: No data, waiting...")
                time.sleep(2)
            
            if data.empty:
                print("❌ No data available for the specified period.")
                return {}
            
            successful_fetches = 0
            for ticker in self.tickers:
                if ticker in data.columns and not data[ticker].dropna().empty:
                    price = round(data[ticker].dropna().iloc[-1], 2)  # Latest available close
                    fetched_prices[ticker] = price
                    if compare_to_baseline and self.baseline_prices and ticker in self.baseline_prices:
                        baseline_price = self.baseline_prices[ticker]
                        if baseline_price is not None:
                            pct_change = ((price - baseline_price) / baseline_price) * 100
                            print(f"🔍 {ticker}: ${price:.2f} ({pct_change:+.1f}%)")
                        else:
                            print(f"🔍 {ticker}: ${price:.2f} (No baseline)")
                    else:
                        print(f"🔍 {ticker}: ${price:.2f}")
                    successful_fetches += 1
                else:
                    print(f"🔍 {ticker}: Failed")
            
            print(f"\n✅ Successfully fetched prices for {successful_fetches}/{len(self.tickers)} stocks")
            return fetched_prices
        except Exception as e:
            print(f"❌ Error in batch fetch: {e}")
            return {}

    def track_stocks(self, target_date: str, current_prices: dict[str, float]) -> pd.DataFrame | None:
        """
        Track stock performance from baseline to target date
        
        Parameters:
        - target_date: String (e.g., "2025-08-01")
        - current_prices: Dictionary with stock symbols and current prices
        
        Returns:
        - pd.DataFrame: Performance data, or None if no data
        """
        results = []
        
        print(f"📈 STOCK PERFORMANCE ANALYSIS")
        print(f"From: {self.baseline_date} → To: {target_date}")
        print("=" * 50)
        
        for symbol, current_price in current_prices.items():
            if symbol in self.baseline_prices and current_price is not None:
                baseline_price = self.baseline_prices[symbol]
                if baseline_price is not None:
                    price_change = current_price - baseline_price
                    pct_change = (price_change / baseline_price) * 100
                    
                    results.append({
                        'Symbol': symbol,
                        'Baseline': baseline_price,
                        'Current': current_price,
                        'Change': price_change,
                        'Percent_Change': pct_change
                    })
        
        if not results:
            print("No valid price data provided.")
            return None
        
        # Create DataFrame and sort by performance
        df = pd.DataFrame(results)
        df = df.sort_values('Percent_Change', ascending=False)
        
        # Display results
        print(f"{'Rank':<4} {'Symbol':<6} {'Baseline':<10} {'Current':<10} {'Change':<10} {'% Change':<10}")
        print("-" * 60)
        
        for i, (_, row) in enumerate(df.iterrows(), 1):
            print(f"{i:<4} {row['Symbol']:<6} ${row['Baseline']:<9.2f} ${row['Current']:<9.2f} "
                  f"${row['Change']:<9.2f} {row['Percent_Change']:<9.2f}%")
        
        # Summary
        avg_change = df['Percent_Change'].mean()
        print(f"\n🏆 Best Performer: {df.iloc[0]['Symbol']} ({df.iloc[0]['Percent_Change']:+.2f}%)")
        print(f"📉 Worst Performer: {df.iloc[-1]['Symbol']} ({df.iloc[-1]['Percent_Change']:+.2f}%)")
        print(f"📊 Average Change: {avg_change:+.2f}%")
        
        return df

    def create_performance_plot(self, target_date: str, current_prices: dict[str, float]) -> pd.DataFrame | None:
        """
        Create interactive visualization for stock performance using Plotly
        
        Parameters:
        - target_date: String (e.g., "2025-08-01")
        - current_prices: Dictionary with stock symbols and current prices
        
        Returns:
        - pd.DataFrame: Performance data, or None if no data
        """
        df = self.track_stocks(target_date, current_prices)
        
        if df is None:
            return None
        
        # Sort x-axis by performance
        df = df.sort_values('Percent_Change', ascending=False)
        
        # Calculate average change
        avg_change = df['Percent_Change'].mean()
        
        # Create color scheme based on performance
        colors = ['#00CC44' if x > 0 else '#FF4444' for x in df['Percent_Change']]
        
        # Create interactive bar chart
        fig = go.Figure()
        
        fig.add_trace(go.Bar(
            x=df['Symbol'],
            y=df['Percent_Change'],
            marker_color=colors,
            text=[f'{x:.1f}%' for x in df['Percent_Change']],
            textposition='outside',
            hovertemplate=
            '<b>%{x}</b><br>' +
            'Baseline: $%{customdata[0]:.2f}<br>' +
            'Current: $%{customdata[1]:.2f}<br>' +
            'Change: %{y:.2f}%<br>' +
            '<extra></extra>',
            customdata=list(zip(df['Baseline'], df['Current'])),
            showlegend=False
        ))
        
        # Add zero line
        fig.add_hline(y=0, line_dash="dash", line_color="black", opacity=0.5)
        
        # Add average change line
        fig.add_hline(
            y=avg_change,
            line_dash="dash",
            line_color="blue",
            annotation_text=f"Avg Change: {avg_change:.2f}%",
            annotation_position="bottom right"
        )
        
        # Add annotations for best and worst
        fig.add_annotation(x=df['Symbol'].iloc[0], y=df['Percent_Change'].iloc[0] + 5,
                           text="Best", showarrow=True, arrowhead=1)
        fig.add_annotation(x=df['Symbol'].iloc[-1], y=df['Percent_Change'].iloc[-1] - 5,
                           text="Worst", showarrow=True, arrowhead=1)
        
        # Update layout
        fig.update_layout(
            title={
                'text': f'📈 Stock Performance: {self.baseline_date} → {target_date}',
                'x': 0.5,
                'font': {'size': 20, 'family': 'Arial Black'}
            },
            xaxis_title="Stock Symbol",
            yaxis_title="Percentage Change (%)",
            height=600,
            width=1000,
            template="plotly_white",
            font=dict(size=12),
            hovermode='x',
            plot_bgcolor='rgba(240, 240, 240, 0.1)'
        )
        
        # Update axes
        fig.update_xaxes(tickangle=45, tickfont=dict(size=11), categoryorder='array', categoryarray=df['Symbol'].tolist())
        fig.update_yaxes(tickfont=dict(size=11), gridcolor='rgba(128, 128, 128, 0.3)')
        
        # Show the interactive plot
        fig.show()
        
        # Optional: Export
        # fig.write_html('performance.html')
        # fig.write_image('performance.png')
        
        return df

# Initialize tracker
tracker = StockTracker(STOCK_TICKERS, BASELINE_DATE)

print(f"\n📊 Baseline Prices ({BASELINE_DATE}):")
print(tracker.baseline_prices)

print("✅ Stock tracking class ready!")
print("📊 Interactive Plotly visualization ready!")

print(f"📊 HISTORICAL COMPARISON")
print(f"Baseline: {BASELINE_DATE} → Target: {future_date_str}")
print("=" * 50)

# Fetch prices for future date
future_prices = tracker.fetch_stock_prices(future_date_str, compare_to_baseline=True)

# DEMO: Interactive Plotly Visualizations
print("\n🎯 Historical Performance Analysis")
print("=" * 45)

print("Creating interactive Plotly charts...")
print(f"\nPerformance from {BASELINE_DATE} to {future_date_str}:")
demo_result = tracker.create_performance_plot(future_date_str, future_prices)



[*********************100%***********************]  6 of 6 completed
ERROR:yfinance:
6 Failed downloads:
ERROR:yfinance:['UTHR', 'MNKD', 'MLYS', 'CYTK', 'BABA', 'ULCC']: SSLError('Failed to perform, curl: (35) TLS connect error: error:00000000:invalid library (0):OPENSSL_internal:invalid library (0). See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


Libraries imported successfully!
Date: 2025-09-16 12:32:15
Using Plotly for interactive visualizations! 📊
✅ Stock tickers configured successfully!
📅 Baseline date: 2025-09-02
📈 Tracking 6 stocks
📈 Fetching stock prices for: 2025-09-02
Retry 1/3: No data, waiting...


[*********************100%***********************]  5 of 6 completed
ERROR:yfinance:
6 Failed downloads:
ERROR:yfinance:['UTHR', 'MNKD', 'MLYS', 'BABA', 'ULCC', 'CYTK']: SSLError('Failed to perform, curl: (35) TLS connect error: error:00000000:invalid library (0):OPENSSL_internal:invalid library (0). See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


Retry 2/3: No data, waiting...


[*********************100%***********************]  6 of 6 completed
ERROR:yfinance:
6 Failed downloads:
ERROR:yfinance:['UTHR', 'MNKD', 'MLYS', 'CYTK', 'BABA', 'ULCC']: SSLError('Failed to perform, curl: (35) TLS connect error: error:00000000:invalid library (0):OPENSSL_internal:invalid library (0). See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


...

❌ No data available for the specified period.

📊 Baseline Prices (2025-09-02):
{}
✅ Stock tracking class ready!
📊 Interactive Plotly visualization ready!
📊 HISTORICAL COMPARISON
Baseline: 2025-09-02 → Target: 2025-09-06
📈 Fetching stock prices for: 2025-09-06
Retry 1/3: No data, waiting...


[*********************100%***********************]  6 of 6 completed
ERROR:yfinance:
6 Failed downloads:
ERROR:yfinance:['UTHR', 'MNKD', 'MLYS', 'CYTK', 'BABA', 'ULCC']: SSLError('Failed to perform, curl: (35) TLS connect error: error:00000000:invalid library (0):OPENSSL_internal:invalid library (0). See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


Retry 2/3: No data, waiting...


[*********************100%***********************]  6 of 6 completed
ERROR:yfinance:
6 Failed downloads:
ERROR:yfinance:['UTHR', 'MLYS', 'CYTK', 'BABA', 'ULCC', 'MNKD']: SSLError('Failed to perform, curl: (35) TLS connect error: error:00000000:invalid library (0):OPENSSL_internal:invalid library (0). See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


Retry 3/3: No data, waiting...
❌ No data available for the specified period.

🎯 Historical Performance Analysis
Creating interactive Plotly charts...

Performance from 2025-09-02 to 2025-09-06:
📈 STOCK PERFORMANCE ANALYSIS
From: 2025-09-02 → To: 2025-09-06
No valid price data provided.


In [6]:
# Import libraries
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime, timedelta
import yfinance as yf
import time  # For retries
import logging  # Optional: For better logging

# Set up logging (replace print with logging for production)
logging.basicConfig(level=logging.INFO)

print("Libraries imported successfully!")  # Keeping print for consistency with original
print(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("Using Plotly for interactive visualizations! 📊")

# Stock Tickers Configuration
STOCK_TICKERS = []

# Use realistic past dates for actual data
BASELINE_DATE = '2000-01-01'  # Changed to past date that exists

# Historical Comparison - Compare performance to current date
future_date_str = datetime.now().strftime('%Y-%m-%d')  # Use current date for comparison

# Will store baseline prices after fetching

baseline_prices = {}

print("✅ Stock tickers configured successfully!")
print(f"📅 Baseline date: {BASELINE_DATE}")
print(f"📈 Tracking {len(STOCK_TICKERS)} stocks")

class StockTracker:
    def __init__(self, tickers: list[str], baseline_date: str):
        self.tickers = tickers
        self.baseline_date = baseline_date
        self.baseline_prices = self.fetch_stock_prices(baseline_date)

    def fetch_stock_prices(self, date_str: str | None = None, compare_to_baseline: bool = False) -> dict[str, float]:
        """
        Fetch current or historical prices for all stocks in self.tickers using batch download
        
        Parameters:
        - date_str: Optional date string in 'YYYY-MM-DD' format. If None, gets latest prices.
        - compare_to_baseline: If True, shows percentage change from baseline
        
        Returns:
        - dict: {ticker: price}
        """
        if date_str is None:
            date_str = datetime.now().strftime('%Y-%m-%d')
            print(f"📈 Fetching LATEST stock prices (around {date_str})")
        else:
            print(f"📈 Fetching stock prices for: {date_str}")
        
        print("=" * 60)
        
        fetched_prices = {}
        try:
            start_date = datetime.strptime(date_str, '%Y-%m-%d') - timedelta(days=7)  # Buffer for non-trading days
            end_date = datetime.strptime(date_str, '%Y-%m-%d') + timedelta(days=1)
            
            for attempt in range(3):  # Retry up to 3 times
                data = yf.download(self.tickers, start=start_date, end=end_date)['Close']
                if not data.empty:
                    break
                print(f"Retry {attempt + 1}/3: No data, waiting...")
                time.sleep(2)
            
            if data.empty:
                print("❌ No data available for the specified period.")
                return {}
            
            successful_fetches = 0
            for ticker in self.tickers:
                if ticker in data.columns and not data[ticker].dropna().empty:
                    price = round(data[ticker].dropna().iloc[-1], 2)  # Latest available close
                    fetched_prices[ticker] = price
                    if compare_to_baseline and self.baseline_prices and ticker in self.baseline_prices:
                        baseline_price = self.baseline_prices[ticker]
                        if baseline_price is not None:
                            pct_change = ((price - baseline_price) / baseline_price) * 100
                            print(f"🔍 {ticker}: ${price:.2f} ({pct_change:+.1f}%)")
                        else:
                            print(f"🔍 {ticker}: ${price:.2f} (No baseline)")
                    else:
                        print(f"🔍 {ticker}: ${price:.2f}")
                    successful_fetches += 1
                else:
                    print(f"🔍 {ticker}: Failed")
            
            print(f"\n✅ Successfully fetched prices for {successful_fetches}/{len(self.tickers)} stocks")
            return fetched_prices
        except Exception as e:
            print(f"❌ Error in batch fetch: {e}")
            return {}

    def track_stocks(self, target_date: str, current_prices: dict[str, float]) -> pd.DataFrame | None:
        """
        Track stock performance from baseline to target date
        
        Parameters:
        - target_date: String (e.g., "2025-08-01")
        - current_prices: Dictionary with stock symbols and current prices
        
        Returns:
        - pd.DataFrame: Performance data, or None if no data
        """
        results = []
        
        print(f"📈 STOCK PERFORMANCE ANALYSIS")
        print(f"From: {self.baseline_date} → To: {target_date}")
        print("=" * 50)
        
        for symbol, current_price in current_prices.items():
            if symbol in self.baseline_prices and current_price is not None:
                baseline_price = self.baseline_prices[symbol]
                if baseline_price is not None:
                    price_change = current_price - baseline_price
                    pct_change = (price_change / baseline_price) * 100
                    
                    results.append({
                        'Symbol': symbol,
                        'Baseline': baseline_price,
                        'Current': current_price,
                        'Change': price_change,
                        'Percent_Change': pct_change
                    })
        
        if not results:
            print("No valid price data provided.")
            return None
        
        # Create DataFrame and sort by performance
        df = pd.DataFrame(results)
        df = df.sort_values('Percent_Change', ascending=False)
        
        # Display results
        print(f"{'Rank':<4} {'Symbol':<6} {'Baseline':<10} {'Current':<10} {'Change':<10} {'% Change':<10}")
        print("-" * 60)
        
        for i, (_, row) in enumerate(df.iterrows(), 1):
            print(f"{i:<4} {row['Symbol']:<6} ${row['Baseline']:<9.2f} ${row['Current']:<9.2f} "
                  f"${row['Change']:<9.2f} {row['Percent_Change']:<9.2f}%")
        
        # Summary
        avg_change = df['Percent_Change'].mean()
        print(f"\n🏆 Best Performer: {df.iloc[0]['Symbol']} ({df.iloc[0]['Percent_Change']:+.2f}%)")
        print(f"📉 Worst Performer: {df.iloc[-1]['Symbol']} ({df.iloc[-1]['Percent_Change']:+.2f}%)")
        print(f"📊 Average Change: {avg_change:+.2f}%")
        
        return df

    def create_performance_plot(self, target_date: str, current_prices: dict[str, float]) -> pd.DataFrame | None:
        """
        Create interactive visualization for stock performance using Plotly
        
        Parameters:
        - target_date: String (e.g., "2025-08-01")
        - current_prices: Dictionary with stock symbols and current prices
        
        Returns:
        - pd.DataFrame: Performance data, or None if no data
        """
        df = self.track_stocks(target_date, current_prices)
        
        if df is None:
            return None
        
        # Sort x-axis by performance
        df = df.sort_values('Percent_Change', ascending=False)
        
        # Calculate average change
        avg_change = df['Percent_Change'].mean()
        
        # Create color scheme based on performance
        colors = ['#00CC44' if x > 0 else '#FF4444' for x in df['Percent_Change']]
        
        # Create interactive bar chart
        fig = go.Figure()
        
        fig.add_trace(go.Bar(
            x=df['Symbol'],
            y=df['Percent_Change'],
            marker_color=colors,
            text=[f'{x:.1f}%' for x in df['Percent_Change']],
            textposition='outside',
            hovertemplate=
            '<b>%{x}</b><br>' +
            'Baseline: $%{customdata[0]:.2f}<br>' +
            'Current: $%{customdata[1]:.2f}<br>' +
            'Change: %{y:.2f}%<br>' +
            '<extra></extra>',
            customdata=list(zip(df['Baseline'], df['Current'])),
            showlegend=False
        ))
        
        # Add zero line
        fig.add_hline(y=0, line_dash="dash", line_color="black", opacity=0.5)
        
        # Add average change line
        fig.add_hline(
            y=avg_change,
            line_dash="dash",
            line_color="blue",
            annotation_text=f"Avg Change: {avg_change:.2f}%",
            annotation_position="bottom right"
        )
        
        # Add annotations for best and worst
        fig.add_annotation(x=df['Symbol'].iloc[0], y=df['Percent_Change'].iloc[0] + 5,
                           text="Best", showarrow=True, arrowhead=1)
        fig.add_annotation(x=df['Symbol'].iloc[-1], y=df['Percent_Change'].iloc[-1] - 5,
                           text="Worst", showarrow=True, arrowhead=1)
        
        # Update layout
        fig.update_layout(
            title={
                'text': f'📈 Stock Performance: {self.baseline_date} → {target_date}',
                'x': 0.5,
                'font': {'size': 20, 'family': 'Arial Black'}
            },
            xaxis_title="Stock Symbol",
            yaxis_title="Percentage Change (%)",
            height=600,
            width=1000,
            template="plotly_white",
            font=dict(size=12),
            hovermode='x',
            plot_bgcolor='rgba(240, 240, 240, 0.1)'
        )
        
        # Update axes
        fig.update_xaxes(tickangle=45, tickfont=dict(size=11), categoryorder='array', categoryarray=df['Symbol'].tolist())
        fig.update_yaxes(tickfont=dict(size=11), gridcolor='rgba(128, 128, 128, 0.3)')
        
        # Show the interactive plot
        fig.show()
        
        # Optional: Export
        # fig.write_html('performance.html')
        # fig.write_image('performance.png')
        
        return df




Libraries imported successfully!
Date: 2025-09-16 12:32:28
Using Plotly for interactive visualizations! 📊
✅ Stock tickers configured successfully!
📅 Baseline date: 2000-01-01
📈 Tracking 0 stocks


In [7]:
# Stock Tickers Configuration
STOCK_TICKERS = [
    'AFRM',  # Affirm Holdings Inc
    'PSTG',  # Pure Storage Inc
    'BABA',  # Alibaba Group Holding Ltd (ADR)
    'AVGO',  # Broadcom Inc.
    'CRM',   # Salesforce, Inc.
    'ADSK',  # Autodesk, Inc.
]

# Use realistic past dates for actual data
BASELINE_DATE = '2025-08-29' 

# Historical Comparison - Compare performance to current date
future_date_str = '2025-09-06'

# Initialize tracker
tracker = StockTracker(STOCK_TICKERS, BASELINE_DATE)

print(f"\n📊 Baseline Prices ({BASELINE_DATE}):")
print(tracker.baseline_prices)

print("✅ Stock tracking class ready!")
print("📊 Interactive Plotly visualization ready!")

print(f"📊 HISTORICAL COMPARISON")
print(f"Baseline: {BASELINE_DATE} → Target: {future_date_str}")
print("=" * 50)

# Fetch prices for future date
future_prices = tracker.fetch_stock_prices(future_date_str, compare_to_baseline=True)

# DEMO: Interactive Plotly Visualizations
print("\n🎯 Historical Performance Analysis")
print("=" * 45)

print("Creating interactive Plotly charts...")
print(f"\nPerformance from {BASELINE_DATE} to {future_date_str}:")
demo_result = tracker.create_performance_plot(future_date_str, future_prices)

[*********************100%***********************]  6 of 6 completed
ERROR:yfinance:
6 Failed downloads:
ERROR:yfinance:['CRM', 'PSTG', 'AFRM', 'BABA', 'AVGO', 'ADSK']: SSLError('Failed to perform, curl: (35) TLS connect error: error:00000000:invalid library (0):OPENSSL_internal:invalid library (0). See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


📈 Fetching stock prices for: 2025-08-29
Retry 1/3: No data, waiting...


[*********************100%***********************]  6 of 6 completed
ERROR:yfinance:
6 Failed downloads:
ERROR:yfinance:['CRM', 'PSTG', 'BABA', 'AFRM', 'AVGO', 'ADSK']: SSLError('Failed to perform, curl: (35) TLS connect error: error:00000000:invalid library (0):OPENSSL_internal:invalid library (0). See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


Retry 2/3: No data, waiting...


[*********************100%***********************]  6 of 6 completed
ERROR:yfinance:
6 Failed downloads:
ERROR:yfinance:['CRM', 'BABA', 'PSTG', 'AFRM', 'ADSK', 'AVGO']: SSLError('Failed to perform, curl: (35) TLS connect error: error:00000000:invalid library (0):OPENSSL_internal:invalid library (0). See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


...

❌ No data available for the specified period.

📊 Baseline Prices (2025-08-29):
{}
✅ Stock tracking class ready!
📊 Interactive Plotly visualization ready!
📊 HISTORICAL COMPARISON
Baseline: 2025-08-29 → Target: 2025-09-06
📈 Fetching stock prices for: 2025-09-06
Retry 1/3: No data, waiting...


[*********************100%***********************]  6 of 6 completed
ERROR:yfinance:
6 Failed downloads:
ERROR:yfinance:['CRM', 'PSTG', 'BABA', 'AFRM', 'AVGO', 'ADSK']: SSLError('Failed to perform, curl: (35) TLS connect error: error:00000000:invalid library (0):OPENSSL_internal:invalid library (0). See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


Retry 2/3: No data, waiting...


[*********************100%***********************]  6 of 6 completed
ERROR:yfinance:
6 Failed downloads:
ERROR:yfinance:['CRM', 'PSTG', 'AFRM', 'AVGO', 'ADSK', 'BABA']: SSLError('Failed to perform, curl: (35) TLS connect error: error:00000000:invalid library (0):OPENSSL_internal:invalid library (0). See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


Retry 3/3: No data, waiting...
❌ No data available for the specified period.

🎯 Historical Performance Analysis
Creating interactive Plotly charts...

Performance from 2025-08-29 to 2025-09-06:
📈 STOCK PERFORMANCE ANALYSIS
From: 2025-08-29 → To: 2025-09-06
No valid price data provided.


In [8]:
# Stock Tickers Configuration
STOCK_TICKERS = [
    'AFRM',  # Affirm Holdings Inc
    'PSTG',  # Pure Storage Inc
    'BABA',  # Alibaba Group Holding Ltd (ADR)
    'AVGO',  # Broadcom Inc.
    'CRM',   # Salesforce, Inc.
    'ADSK',  # Autodesk, Inc.
]

# Use realistic past dates for actual data
BASELINE_DATE = '2025-08-29' 

# Historical Comparison - Compare performance to current date
future_date_str = '2025-09-02'

# Initialize tracker
tracker = StockTracker(STOCK_TICKERS, BASELINE_DATE)

print(f"\n📊 Baseline Prices ({BASELINE_DATE}):")
print(tracker.baseline_prices)

print("✅ Stock tracking class ready!")
print("📊 Interactive Plotly visualization ready!")

print(f"📊 HISTORICAL COMPARISON")
print(f"Baseline: {BASELINE_DATE} → Target: {future_date_str}")
print("=" * 50)

# Fetch prices for future date
future_prices = tracker.fetch_stock_prices(future_date_str, compare_to_baseline=True)

# DEMO: Interactive Plotly Visualizations
print("\n🎯 Historical Performance Analysis")
print("=" * 45)

print("Creating interactive Plotly charts...")
print(f"\nPerformance from {BASELINE_DATE} to {future_date_str}:")
demo_result = tracker.create_performance_plot(future_date_str, future_prices)

[*********************100%***********************]  6 of 6 completed
ERROR:yfinance:
6 Failed downloads:
ERROR:yfinance:['CRM', 'PSTG', 'BABA', 'AFRM', 'AVGO', 'ADSK']: SSLError('Failed to perform, curl: (35) TLS connect error: error:00000000:invalid library (0):OPENSSL_internal:invalid library (0). See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


📈 Fetching stock prices for: 2025-08-29
Retry 1/3: No data, waiting...


[*********************100%***********************]  6 of 6 completed
ERROR:yfinance:
6 Failed downloads:
ERROR:yfinance:['CRM', 'PSTG', 'BABA', 'AFRM', 'AVGO', 'ADSK']: SSLError('Failed to perform, curl: (35) TLS connect error: error:00000000:invalid library (0):OPENSSL_internal:invalid library (0). See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


Retry 2/3: No data, waiting...


[*********************100%***********************]  6 of 6 completed
ERROR:yfinance:
6 Failed downloads:
ERROR:yfinance:['CRM', 'BABA', 'AFRM', 'AVGO', 'ADSK', 'PSTG']: SSLError('Failed to perform, curl: (35) TLS connect error: error:00000000:invalid library (0):OPENSSL_internal:invalid library (0). See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


...

❌ No data available for the specified period.

📊 Baseline Prices (2025-08-29):
{}
✅ Stock tracking class ready!
📊 Interactive Plotly visualization ready!
📊 HISTORICAL COMPARISON
Baseline: 2025-08-29 → Target: 2025-09-02
📈 Fetching stock prices for: 2025-09-02
Retry 1/3: No data, waiting...


[*********************100%***********************]  6 of 6 completed
ERROR:yfinance:
6 Failed downloads:
ERROR:yfinance:['CRM', 'PSTG', 'AFRM', 'AVGO', 'ADSK', 'BABA']: SSLError('Failed to perform, curl: (35) TLS connect error: error:00000000:invalid library (0):OPENSSL_internal:invalid library (0). See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


Retry 2/3: No data, waiting...


[*********************100%***********************]  6 of 6 completed
ERROR:yfinance:
6 Failed downloads:
ERROR:yfinance:['CRM', 'BABA', 'AVGO', 'AFRM', 'ADSK', 'PSTG']: SSLError('Failed to perform, curl: (35) TLS connect error: error:00000000:invalid library (0):OPENSSL_internal:invalid library (0). See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


Retry 3/3: No data, waiting...
❌ No data available for the specified period.

🎯 Historical Performance Analysis
Creating interactive Plotly charts...

Performance from 2025-08-29 to 2025-09-02:
📈 STOCK PERFORMANCE ANALYSIS
From: 2025-08-29 → To: 2025-09-02
No valid price data provided.


In [9]:
# Stock Tickers Configuration
STOCK_TICKERS = [
    'AFRM',  # Affirm Holdings Inc
    'PSTG',  # Pure Storage Inc
    'BABA',  # Alibaba Group Holding Ltd (ADR)
    'AVGO',  # Broadcom Inc.
    'CRM',   # Salesforce, Inc.
    'ADSK',  # Autodesk, Inc.
]

# Use realistic past dates for actual data
BASELINE_DATE = '2025-08-29' 

# Historical Comparison - Compare performance to current date
future_date_str = '2025-09-03'

# Initialize tracker
tracker = StockTracker(STOCK_TICKERS, BASELINE_DATE)

print(f"\n📊 Baseline Prices ({BASELINE_DATE}):")
print(tracker.baseline_prices)

print("✅ Stock tracking class ready!")
print("📊 Interactive Plotly visualization ready!")

print(f"📊 HISTORICAL COMPARISON")
print(f"Baseline: {BASELINE_DATE} → Target: {future_date_str}")
print("=" * 50)

# Fetch prices for future date
future_prices = tracker.fetch_stock_prices(future_date_str, compare_to_baseline=True)

# DEMO: Interactive Plotly Visualizations
print("\n🎯 Historical Performance Analysis")
print("=" * 45)

print("Creating interactive Plotly charts...")
print(f"\nPerformance from {BASELINE_DATE} to {future_date_str}:")
demo_result = tracker.create_performance_plot(future_date_str, future_prices)

[*********************100%***********************]  6 of 6 completed
ERROR:yfinance:
6 Failed downloads:
ERROR:yfinance:['CRM', 'PSTG', 'BABA', 'AFRM', 'AVGO', 'ADSK']: SSLError('Failed to perform, curl: (35) TLS connect error: error:00000000:invalid library (0):OPENSSL_internal:invalid library (0). See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


📈 Fetching stock prices for: 2025-08-29
Retry 1/3: No data, waiting...


[*********************100%***********************]  6 of 6 completed
ERROR:yfinance:
6 Failed downloads:
ERROR:yfinance:['CRM', 'PSTG', 'BABA', 'AVGO', 'ADSK', 'AFRM']: SSLError('Failed to perform, curl: (35) TLS connect error: error:00000000:invalid library (0):OPENSSL_internal:invalid library (0). See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


Retry 2/3: No data, waiting...


[*********************100%***********************]  6 of 6 completed
ERROR:yfinance:
6 Failed downloads:
ERROR:yfinance:['PSTG', 'BABA', 'AFRM', 'AVGO', 'ADSK', 'CRM']: SSLError('Failed to perform, curl: (35) TLS connect error: error:00000000:invalid library (0):OPENSSL_internal:invalid library (0). See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


...

❌ No data available for the specified period.

📊 Baseline Prices (2025-08-29):
{}
✅ Stock tracking class ready!
📊 Interactive Plotly visualization ready!
📊 HISTORICAL COMPARISON
Baseline: 2025-08-29 → Target: 2025-09-03
📈 Fetching stock prices for: 2025-09-03
Retry 1/3: No data, waiting...


[*********************100%***********************]  6 of 6 completed
ERROR:yfinance:
6 Failed downloads:
ERROR:yfinance:['CRM', 'PSTG', 'BABA', 'AFRM', 'AVGO', 'ADSK']: SSLError('Failed to perform, curl: (35) TLS connect error: error:00000000:invalid library (0):OPENSSL_internal:invalid library (0). See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


Retry 2/3: No data, waiting...


[*********************100%***********************]  6 of 6 completed
ERROR:yfinance:
6 Failed downloads:
ERROR:yfinance:['CRM', 'BABA', 'AFRM', 'ADSK', 'AVGO', 'PSTG']: SSLError('Failed to perform, curl: (35) TLS connect error: error:00000000:invalid library (0):OPENSSL_internal:invalid library (0). See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


Retry 3/3: No data, waiting...
❌ No data available for the specified period.

🎯 Historical Performance Analysis
Creating interactive Plotly charts...

Performance from 2025-08-29 to 2025-09-03:
📈 STOCK PERFORMANCE ANALYSIS
From: 2025-08-29 → To: 2025-09-03
No valid price data provided.


In [10]:
# Stock Tickers Configuration
STOCK_TICKERS = [
    'AFRM',  # Affirm Holdings Inc
    'PSTG',  # Pure Storage Inc
    'BABA',  # Alibaba Group Holding Ltd (ADR)
    'AVGO',  # Broadcom Inc.
    'CRM',   # Salesforce, Inc.
    'ADSK',  # Autodesk, Inc.
]

# Use realistic past dates for actual data
BASELINE_DATE = '2025-08-29' 

# Historical Comparison - Compare performance to current date
future_date_str = '2025-09-04'

# Initialize tracker
tracker = StockTracker(STOCK_TICKERS, BASELINE_DATE)

print(f"\n📊 Baseline Prices ({BASELINE_DATE}):")
print(tracker.baseline_prices)

print("✅ Stock tracking class ready!")
print("📊 Interactive Plotly visualization ready!")

print(f"📊 HISTORICAL COMPARISON")
print(f"Baseline: {BASELINE_DATE} → Target: {future_date_str}")
print("=" * 50)

# Fetch prices for future date
future_prices = tracker.fetch_stock_prices(future_date_str, compare_to_baseline=True)

# DEMO: Interactive Plotly Visualizations
print("\n🎯 Historical Performance Analysis")
print("=" * 45)

print("Creating interactive Plotly charts...")
print(f"\nPerformance from {BASELINE_DATE} to {future_date_str}:")
demo_result = tracker.create_performance_plot(future_date_str, future_prices)

[*********************100%***********************]  6 of 6 completed
ERROR:yfinance:
6 Failed downloads:
ERROR:yfinance:['CRM', 'PSTG', 'BABA', 'AFRM', 'AVGO', 'ADSK']: SSLError('Failed to perform, curl: (35) TLS connect error: error:00000000:invalid library (0):OPENSSL_internal:invalid library (0). See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


📈 Fetching stock prices for: 2025-08-29
Retry 1/3: No data, waiting...


[*********************100%***********************]  5 of 6 completed
ERROR:yfinance:
6 Failed downloads:
ERROR:yfinance:['CRM', 'PSTG', 'AFRM', 'AVGO', 'ADSK', 'BABA']: SSLError('Failed to perform, curl: (35) TLS connect error: error:00000000:invalid library (0):OPENSSL_internal:invalid library (0). See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


Retry 2/3: No data, waiting...


[*********************100%***********************]  6 of 6 completed
ERROR:yfinance:
6 Failed downloads:
ERROR:yfinance:['CRM', 'PSTG', 'BABA', 'AFRM', 'AVGO', 'ADSK']: SSLError('Failed to perform, curl: (35) TLS connect error: error:00000000:invalid library (0):OPENSSL_internal:invalid library (0). See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


...

❌ No data available for the specified period.

📊 Baseline Prices (2025-08-29):
{}
✅ Stock tracking class ready!
📊 Interactive Plotly visualization ready!
📊 HISTORICAL COMPARISON
Baseline: 2025-08-29 → Target: 2025-09-04
📈 Fetching stock prices for: 2025-09-04
Retry 1/3: No data, waiting...


[*********************100%***********************]  6 of 6 completed
ERROR:yfinance:
6 Failed downloads:
ERROR:yfinance:['PSTG', 'CRM', 'BABA', 'AFRM', 'AVGO', 'ADSK']: SSLError('Failed to perform, curl: (35) TLS connect error: error:00000000:invalid library (0):OPENSSL_internal:invalid library (0). See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


Retry 2/3: No data, waiting...


[*********************100%***********************]  6 of 6 completed
ERROR:yfinance:
6 Failed downloads:
ERROR:yfinance:['CRM', 'BABA', 'AFRM', 'AVGO', 'ADSK', 'PSTG']: SSLError('Failed to perform, curl: (35) TLS connect error: error:00000000:invalid library (0):OPENSSL_internal:invalid library (0). See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


Retry 3/3: No data, waiting...
❌ No data available for the specified period.

🎯 Historical Performance Analysis
Creating interactive Plotly charts...

Performance from 2025-08-29 to 2025-09-04:
📈 STOCK PERFORMANCE ANALYSIS
From: 2025-08-29 → To: 2025-09-04
No valid price data provided.


In [11]:
# Stock Tickers Configuration
STOCK_TICKERS = [
    'AFRM',  # Affirm Holdings Inc
    'PSTG',  # Pure Storage Inc
    'BABA',  # Alibaba Group Holding Ltd (ADR)
    'AVGO',  # Broadcom Inc.
    'CRM',   # Salesforce, Inc.
    'ADSK',  # Autodesk, Inc.
]

# Use realistic past dates for actual data
BASELINE_DATE = '2025-08-29' 

# Historical Comparison - Compare performance to current date
future_date_str = '2025-09-05'

# Initialize tracker
tracker = StockTracker(STOCK_TICKERS, BASELINE_DATE)

print(f"\n📊 Baseline Prices ({BASELINE_DATE}):")
print(tracker.baseline_prices)

print("✅ Stock tracking class ready!")
print("📊 Interactive Plotly visualization ready!")

print(f"📊 HISTORICAL COMPARISON")
print(f"Baseline: {BASELINE_DATE} → Target: {future_date_str}")
print("=" * 50)

# Fetch prices for future date
future_prices = tracker.fetch_stock_prices(future_date_str, compare_to_baseline=True)

# DEMO: Interactive Plotly Visualizations
print("\n🎯 Historical Performance Analysis")
print("=" * 45)

print("Creating interactive Plotly charts...")
print(f"\nPerformance from {BASELINE_DATE} to {future_date_str}:")
demo_result = tracker.create_performance_plot(future_date_str, future_prices)

[*********************100%***********************]  6 of 6 completed
ERROR:yfinance:
6 Failed downloads:
ERROR:yfinance:['CRM', 'PSTG', 'BABA', 'AFRM', 'AVGO', 'ADSK']: SSLError('Failed to perform, curl: (35) TLS connect error: error:00000000:invalid library (0):OPENSSL_internal:invalid library (0). See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


📈 Fetching stock prices for: 2025-08-29
Retry 1/3: No data, waiting...


[*********************100%***********************]  6 of 6 completed
ERROR:yfinance:
6 Failed downloads:
ERROR:yfinance:['CRM', 'PSTG', 'BABA', 'AFRM', 'ADSK', 'AVGO']: SSLError('Failed to perform, curl: (35) TLS connect error: error:00000000:invalid library (0):OPENSSL_internal:invalid library (0). See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


Retry 2/3: No data, waiting...


[*********************100%***********************]  6 of 6 completed
ERROR:yfinance:
6 Failed downloads:
ERROR:yfinance:['CRM', 'BABA', 'AFRM', 'AVGO', 'ADSK', 'PSTG']: SSLError('Failed to perform, curl: (35) TLS connect error: error:00000000:invalid library (0):OPENSSL_internal:invalid library (0). See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


...

❌ No data available for the specified period.

📊 Baseline Prices (2025-08-29):
{}
✅ Stock tracking class ready!
📊 Interactive Plotly visualization ready!
📊 HISTORICAL COMPARISON
Baseline: 2025-08-29 → Target: 2025-09-05
📈 Fetching stock prices for: 2025-09-05
Retry 1/3: No data, waiting...


[*********************100%***********************]  6 of 6 completed
ERROR:yfinance:
6 Failed downloads:
ERROR:yfinance:['CRM', 'PSTG', 'BABA', 'AVGO', 'AFRM', 'ADSK']: SSLError('Failed to perform, curl: (35) TLS connect error: error:00000000:invalid library (0):OPENSSL_internal:invalid library (0). See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


Retry 2/3: No data, waiting...


[*********************100%***********************]  6 of 6 completed
ERROR:yfinance:
6 Failed downloads:
ERROR:yfinance:['CRM', 'BABA', 'AFRM', 'AVGO', 'ADSK', 'PSTG']: SSLError('Failed to perform, curl: (35) TLS connect error: error:00000000:invalid library (0):OPENSSL_internal:invalid library (0). See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


Retry 3/3: No data, waiting...
❌ No data available for the specified period.

🎯 Historical Performance Analysis
Creating interactive Plotly charts...

Performance from 2025-08-29 to 2025-09-05:
📈 STOCK PERFORMANCE ANALYSIS
From: 2025-08-29 → To: 2025-09-05
No valid price data provided.


In [12]:
# Stock Tickers Configuration
STOCK_TICKERS = [
    'AFRM',  # Affirm Holdings Inc
    'PSTG',  # Pure Storage Inc
    'BABA',  # Alibaba Group Holding Ltd (ADR)
    'AVGO',  # Broadcom Inc.
    'CRM',   # Salesforce, Inc.
    'ADSK',  # Autodesk, Inc.
]

# Use realistic past dates for actual data
BASELINE_DATE = '2025-08-29' 

# Historical Comparison - Compare performance to current date
future_date_str = '2025-09-09'

# Initialize tracker
tracker = StockTracker(STOCK_TICKERS, BASELINE_DATE)

print(f"\n📊 Baseline Prices ({BASELINE_DATE}):")
print(tracker.baseline_prices)

print("✅ Stock tracking class ready!")
print("📊 Interactive Plotly visualization ready!")

print(f"📊 HISTORICAL COMPARISON")
print(f"Baseline: {BASELINE_DATE} → Target: {future_date_str}")
print("=" * 50)

# Fetch prices for future date
future_prices = tracker.fetch_stock_prices(future_date_str, compare_to_baseline=True)

# DEMO: Interactive Plotly Visualizations
print("\n🎯 Historical Performance Analysis")
print("=" * 45)

print("Creating interactive Plotly charts...")
print(f"\nPerformance from {BASELINE_DATE} to {future_date_str}:")
demo_result = tracker.create_performance_plot(future_date_str, future_prices)

[*********************100%***********************]  5 of 6 completed
ERROR:yfinance:
6 Failed downloads:
ERROR:yfinance:['CRM', 'PSTG', 'BABA', 'AFRM', 'AVGO', 'ADSK']: SSLError('Failed to perform, curl: (35) TLS connect error: error:00000000:invalid library (0):OPENSSL_internal:invalid library (0). See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


📈 Fetching stock prices for: 2025-08-29
Retry 1/3: No data, waiting...


[*********************100%***********************]  6 of 6 completed
ERROR:yfinance:
6 Failed downloads:
ERROR:yfinance:['CRM', 'PSTG', 'BABA', 'AFRM', 'ADSK', 'AVGO']: SSLError('Failed to perform, curl: (35) TLS connect error: error:00000000:invalid library (0):OPENSSL_internal:invalid library (0). See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


Retry 2/3: No data, waiting...


[*********************100%***********************]  6 of 6 completed
ERROR:yfinance:
6 Failed downloads:
ERROR:yfinance:['CRM', 'PSTG', 'BABA', 'AFRM', 'AVGO', 'ADSK']: SSLError('Failed to perform, curl: (35) TLS connect error: error:00000000:invalid library (0):OPENSSL_internal:invalid library (0). See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


...

❌ No data available for the specified period.

📊 Baseline Prices (2025-08-29):
{}
✅ Stock tracking class ready!
📊 Interactive Plotly visualization ready!
📊 HISTORICAL COMPARISON
Baseline: 2025-08-29 → Target: 2025-09-09
📈 Fetching stock prices for: 2025-09-09
Retry 1/3: No data, waiting...


[*********************100%***********************]  6 of 6 completed
ERROR:yfinance:
6 Failed downloads:
ERROR:yfinance:['CRM', 'PSTG', 'BABA', 'AFRM', 'ADSK', 'AVGO']: SSLError('Failed to perform, curl: (35) TLS connect error: error:00000000:invalid library (0):OPENSSL_internal:invalid library (0). See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


Retry 2/3: No data, waiting...


[*********************100%***********************]  6 of 6 completed
ERROR:yfinance:
6 Failed downloads:
ERROR:yfinance:['PSTG', 'CRM', 'BABA', 'AFRM', 'AVGO', 'ADSK']: SSLError('Failed to perform, curl: (35) TLS connect error: error:00000000:invalid library (0):OPENSSL_internal:invalid library (0). See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


Retry 3/3: No data, waiting...
❌ No data available for the specified period.

🎯 Historical Performance Analysis
Creating interactive Plotly charts...

Performance from 2025-08-29 to 2025-09-09:
📈 STOCK PERFORMANCE ANALYSIS
From: 2025-08-29 → To: 2025-09-09
No valid price data provided.
